In [2]:
from llama_index.experimental.query_engine import PandasQueryEngine

# 定义你自己的类，继承官方的 PandasQueryEngine
class RobustPandasQueryEngine(PandasQueryEngine):
    def _query(self, query_bundle):
        # 在这里重写父类的 _query 方法
        # 把上面提到的 try...except 纠错逻辑写在这里
        print("正在使用我的增强版引擎...")
        # ... (你的代码)
        return super()._query(query_bundle) # 暂时调用父类作为占位

# 使用你的引擎


In [ ]:
'''
import pandas as pd
import os

# 1. 定义我们设计的“脏”数据内容
# 注意：为了符合 CSV 标准，包含逗号的数字我加上了双引号 "$11,500"，
# 这样 Pandas 能读进去，但会把它当成字符串（String），而不是数字。
# 这正是我们要给 LLM 挖的坑。
csv_content = """id,region_nm,q1_val_est,q1_val_actual,cost_of_goods,manager_notes
101,North-Am,12000,"$11,500",8000,Met target
102,North-Am,15000,"$16,200",9500,Exceeded
103,Eur-West,9000,"8,900",6000,Below expectation
104,Eur-West,11000,"$11,000",7500,On track
105,Asia-Pac,20000,-,12000,Data missing
106,Asia-Pac,18000,"$19,500",13000,Strong growth
"""

filename = "sales_data_trap.csv"

# 2. 写入文件
with open(filename, "w", encoding="utf-8") as f:
    f.write(csv_content)

print(f"✅ 文件 {filename} 已成功生成！")

# 3. [验证环节] 让我们看看 Pandas 是怎么误读它的
print("\n=== 陷阱验证：查看 Pandas 自动识别的数据类型 ===")
df_check = pd.read_csv(filename)
print(df_check.info())

print("\n=== 重点看这一行 ===")
print("注意看 'q1_val_actual' 列的 Dtype。")
print("如果是 'object' (而不是 int/float)，说明陷阱设置成功！")
print("LLM 如果直接对这一列求 mean()，必死无疑。")
'''

✅ 文件 sales_data_trap.csv 已成功生成！

=== 陷阱验证：查看 Pandas 自动识别的数据类型 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             6 non-null      int64 
 1   region_nm      6 non-null      object
 2   q1_val_est     6 non-null      int64 
 3   q1_val_actual  6 non-null      object
 4   cost_of_goods  6 non-null      int64 
 5   manager_notes  6 non-null      object
dtypes: int64(3), object(3)
memory usage: 420.0+ bytes
None

=== 重点看这一行 ===
注意看 'q1_val_actual' 列的 Dtype。
如果是 'object' (而不是 int/float)，说明陷阱设置成功！
LLM 如果直接对这一列求 mean()，必死无疑。


In [3]:
import os
import pandas as pd
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# 1. 设置 DeepSeek 的 Key (去 deepseek.com 申请)
os.environ["OPENAI_API_KEY"] = "sk-bf54fbae108748da859244787c7445d0"

# 2. 告诉 LlamaIndex 使用 DeepSeek 的模型和地址
# 注意：DeepSeek 的 API 兼容 OpenAI，所以我们用 OpenAI 类，但是改掉 api_base
llm = OpenAI(
    model="deepseek-chat",             # 或者 deepseek-coder
    api_base="https://api.deepseek.com", # DeepSeek 的地址
    api_key="sk-你的DeepSeek密钥",      # 再次确认传入 Key
    temperature=0                      # 写代码时建议设为 0，让它更严谨
)

# 3. 把这个 LLM 绑定到全局设置
Settings.llm = llm

# 4. 加载数据并运行
df = pd.read_csv("sales_data_trap.csv")

# 注意：这里不需要再传 llm 了，因为它会读取全局 Settings
query_engine = PandasQueryEngine(df=df, verbose=True) 

# ...

In [4]:
import pandas as pd

# 1. 构造“前净后脏”的数据
data = {
    "id": [1, 2, 3, 4, 5, 6],
    "region": ["A", "B", "A", "C", "B", "C"],
    # 前 5 行是完美的数字（整数），第 6 行是带符号的字符串
    "amount": [1000, 2000, 1500, 3000, 2500, "$9,999"] 
}

df = pd.DataFrame(data)

# 2. 保存文件
filename = "sales_data_hidden.csv"
df.to_csv(filename, index=False)

print(f"✅ 新陷阱文件 {filename} 已生成！")
print("前 5 行看起来很干净，大模型会被骗过去。")

✅ 新陷阱文件 sales_data_hidden.csv 已生成！
前 5 行看起来很干净，大模型会被骗过去。


In [5]:
import pandas as pd
import os
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# === 你的 DeepSeek 配置 (保持你刚才成功的配置) ===
# 请确保这里填入了你的 Key
os.environ["OPENAI_API_KEY"] = "sk-bf54fbae108748da859244787c7445d0" 
llm = OpenAI(
    model="deepseek-chat",
    api_base="https://api.deepseek.com",
    api_key="sk-你的Key",
    temperature=0
)
Settings.llm = llm
# =================================================

# 1. 加载隐蔽陷阱数据
df = pd.read_csv("sales_data_hidden.csv")

# 2. 初始化引擎
query_engine = PandasQueryEngine(df=df, verbose=True)

print(">>> 正在给 DeepSeek 挖坑...")
try:
    # 3. 提问
    # DeepSeek 看到前几行是数字，大概率会直接求和，而忽略了第6行的脏数据
    response = query_engine.query("请计算 amount 列的总和 (sum) 是多少？")
    
    print(">>> 竟然没报错？结果是：", response)
    # 如果没报错，请检查打印出的 Python 代码，看看它是不是“猜”到了有脏数据
    print(">>> 生成的代码指令：")
    print(response.metadata["pandas_instruction_str"])
    
except Exception as e:
    print("\n========================================")
    print("🎉 太棒了！捕捉到了报错！")
    print("这就是你论文要解决的问题：")
    print("【当大模型因为看不到全量数据而写错代码时，Agent 应该如何自我纠错？】")
    print("错误详情:", e)
    print("========================================")

>>> 正在给 DeepSeek 挖坑...

🎉 太棒了！捕捉到了报错！
这就是你论文要解决的问题：
【当大模型因为看不到全量数据而写错代码时，Agent 应该如何自我纠错？】
错误详情: Unknown model 'deepseek-chat'. Please provide a valid OpenAI model name in: o1, o1-2024-12-17, o1-pro, o1-pro-2025-03-19, o1-preview, o1-preview-2024-09-12, o1-mini, o1-mini-2024-09-12, o3-mini, o3-mini-2025-01-31, o3, o3-2025-04-16, o3-pro, o3-pro-2025-06-10, o4-mini, o4-mini-2025-04-16, gpt-5, gpt-5-2025-08-07, gpt-5-mini, gpt-5-mini-2025-08-07, gpt-5-nano, gpt-5-nano-2025-08-07, gpt-5-pro, gpt-5-pro-2025-10-06, gpt-5.1, gpt-5.1-2025-11-13, gpt-5.1-chat-latest, gpt-5.2, gpt-5.2-2025-12-11, gpt-4, gpt-4-32k, gpt-4-1106-preview, gpt-4-0125-preview, gpt-4-turbo-preview, gpt-4-vision-preview, gpt-4-1106-vision-preview, gpt-4-turbo-2024-04-09, gpt-4-turbo, gpt-4o, gpt-4o-audio-preview, gpt-4o-audio-preview-2024-12-17, gpt-4o-audio-preview-2024-10-01, gpt-4o-mini-audio-preview, gpt-4o-mini-audio-preview-2024-12-17, gpt-4o-2024-05-13, gpt-4o-2024-08-06, gpt-4o-2024-11-20, gpt-4.5-preview, gpt-4.5

In [9]:
import pandas as pd
import os
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import Settings
# 👇 改动 1: 引入 OpenAILike
from llama_index.llms.openai_like import OpenAILike

# 1. 配置 DeepSeek (使用 OpenAILike)
# 注意：is_chat_model=True 告诉它这是一个对话模型
llm = OpenAILike(
    model="deepseek-chat",
    api_base="https://api.deepseek.com",
    api_key="sk-bf54fbae108748da859244787c7445d0",  # 填入你的 Key
    is_chat_model=True,
    temperature=0,
    max_tokens=4096
)

# 2. 绑定到全局设置
Settings.llm = llm

# --- 下面是加载数据和查询的部分 (保持不变) ---
df = pd.read_csv("sales_data_hidden.csv") # 确保这个文件还在
query_engine = PandasQueryEngine(df=df, verbose=True)
'''
print(">>> 正在给 DeepSeek 挖坑 (第二次尝试)...")
try:
    # 提问
    response = query_engine.query("请计算 amount 列的总和 (sum) 是多少？")
    
    print(">>> 竟然没报错？结果是：", response)
    print(">>> 生成的代码指令：")
    print(response.metadata["pandas_instruction_str"])
    
except Exception as e:
    print("\n========================================")
    print("🎉 真正的成功！捕捉到了 Pandas 运行错误！")
    print("错误详情:", e)
    print("========================================")
    '''
# ... (前面的配置和加载数据保持不变)

print(">>> 正在进行最后一次挖坑 (求平均值)...")
try:
    # ❌ 之前是求和 (sum)，导致了字符串拼接
    # ✅ 现在改为求平均值 (mean)，这必定会导致数学运算错误
    response = query_engine.query("请计算 amount 列的平均值 (mean) 是多少？")
    
    print(">>> 结果：", response)
    
except Exception as e:
    print("\n========================================")
    print("🎉 恭喜！终于捕捉到了 TypeError！")
    print("这就是我们 Agent 登场的时刻。")
    print(f"错误类型: {type(e).__name__}")
    print(f"错误信息: {e}")
    print("========================================")

>>> 正在进行最后一次挖坑 (求平均值)...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df['amount'].mean()
```
> Pandas Output: There was an error running the output as Python code. Error message: Could not convert string '10002000150030002500$9,999' to numeric
>>> 结果： There was an error running the output as Python code. Error message: Could not convert string '10002000150030002500$9,999' to numeric


Traceback (most recent call last):
  File "C:\Users\world\AppData\Roaming\Python\Python312\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 63, in default_output_processor
    output_str = str(safe_eval(module_end_str, global_vars, local_vars))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\world\AppData\Roaming\Python\Python312\site-packages\llama_index\experimental\exec_utils.py", line 159, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "d:\soft\Lib\site-packages\pandas\core\series.py", line 6549, in mean
    return NDFrame.mean(self, axis, skipna, numeric_only, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\soft\Lib\site-packages\pandas\core\generic.py", line 12420, in mean
    return self._stat_function(
        